# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

BFor this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [1]:
## Connect to DB server on AWS
!pip install mysql-connector-python
import mysql.connector
# imports the python package, mysql
import config1
cnx = mysql.connector.connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password
)
cursor = cnx.cursor()


In [2]:
## Create new DB 
cursor.execute("CREATE DATABASE yelp_data")
cnx.database = 'yelp_data'
# this is PYTHON code.
# this allows you to connect to the AWS server.
# flatiron-database.c3qfl5qwsniw.us-east-1.rds.amazonaws.com is what you are connecting to.
# cnx is an OBJECT.

In [3]:
# LIST OF THINGS WE NEED FOR TABLES
# business name, rating, price, review_text, review_date, number_of_reviews
# BUSINESS = business name, business_rating, price, number_of_reviews
# REVIEWS = review_text, review_rating, review_date

In [14]:
# Create a table for the Businesses
create_businesses = """CREATE TABLE business
    (business_id text PRIMARY KEY,
    business_name text,
    business_rating double,
    price varchar(10),
    no_of_reviews int
    );
"""
cursor.execute(create_businesses)

ProgrammingError: 1170 (42000): BLOB/TEXT column 'business_id' used in key specification without a key length

In [15]:
cnx.commit()

In [ ]:
# Create a table for the reviews
create_reviews = """
    )
"""

## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [3]:
# Write a function to make a call to the yelp API
import requests
import json
url = 'https://api.yelp.com/v3/businesses/search'
client_id = 'V7jC9R0wg8iyi0uprqrIDQ'
api_key = '__RrqPgntWvjGhyjuZrEW4sn6o3g-zID39CzFyAO8qkTIUgxGBvDdeB3_v3_qrBeCZ4QyimcSchxfoaQxx9LO-Kff6XNsoXvlFAXZMuOu66CyUS__M7xA4oZsSjpXXYx'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


term = 'Irish'
location = 'Manhattan'
categories = 'bars'
radius = 1000
status = True
offset = 0


url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'categories' : categories,
                'open now' : status,
                'radius' : radius,
                'offset' : offset
            }
response = requests.get(url, headers=headers, params=url_params)



In [4]:
response = requests.get(url, headers=headers, params=url_params)
data = response.json()
print(data['total'])

129


In [5]:
business = data['businesses'][5]
business

{'id': 'vA5yC1MqC7N9gs9q-Kh2nQ',
 'alias': 'm-j-smiths-food-and-spirits-new-york-3',
 'name': "M J  Smith's Food & Spirits",
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/7OVNVyMk-GqMwRFAWQlEaQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/m-j-smiths-food-and-spirits-new-york-3?adjust_creative=V7jC9R0wg8iyi0uprqrIDQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=V7jC9R0wg8iyi0uprqrIDQ',
 'review_count': 27,
 'categories': [{'alias': 'irish_pubs', 'title': 'Irish Pub'},
  {'alias': 'tradamerican', 'title': 'American (Traditional)'}],
 'rating': 4.5,
 'coordinates': {'latitude': 40.75514, 'longitude': -73.9786},
 'transactions': [],
 'location': {'address1': '9 East 45th St',
  'address2': None,
  'address3': '',
  'city': 'New York',
  'zip_code': '10017',
  'country': 'US',
  'state': 'NY',
  'display_address': ['9 East 45th St', 'New York, NY 10017']},
 'phone': '+16465967497',
 'display_phone': '(646) 596-7497',
 'distance': 551.963561

In [10]:
def parse_business(offset):
    
    url = 'https://api.yelp.com/v3/businesses/search'
    term = 'Irish'
    location = 'Manhattan'
    categories = 'bars'
    radius = 1000
    status = True


    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'categories' : categories,
                    'open now' : status,
                    'radius' : radius,
                    'offset' : offset
                }
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    print(data['total'])
    businesses = data['businesses']
    # create a container to hold our parsed data
    parsed_data = []
    bus_objs = []
    
    # loop through our business and parse each individual business
    for business in businesses:
        if 'price' not in business:
            business['price'] = ""
        biz_tuple = (business['id'],
                    business['name'], 
                    business['price'],
                    business['rating'],
                    business['review_count'])
        parsed_data.append(biz_tuple)
        b = Business(business['id'],  business['name'], business['price'], 
                        business['rating'], business['review_count'])
        bus_objs.append(b)
        
    return bus_objs
    # in the loop, add each individual parsed parsed business to our data container
    
    # return the container with all of the parsed results

In [11]:
b_objs = parse_business(0)
b_objs

129


In [12]:
b = b_objs[0]
print(f"Id: {b.bus_id} name: {b.name} price: {b.price} rating: {b.rating} review count: {b.review_count}")

Id: c8FID4VcTtDDONokhACvfQ name: O'Lunney's price: $$ rating: 4.0 review count: 382


In [13]:
def parse_review(b_obj):
    
    business_id = b_obj.bus_id
    url = 'https://api.yelp.com/v3/businesses/' + business_id + '/reviews'


    url_params = {}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    reviews = data['reviews']
    # create a container to hold our parsed data
    parsed_data = []
    
    # loop through our business and parse each individual business
    for review in reviews:
        review_tuple = (review['id'], 
                    review['text'],
                    review['rating'],
                    review['time_created'])
        parsed_data.append(review_tuple)
        
        review_obj = Review(review['id'], 
                    review['text'],
                    review['rating'],
                    review['time_created'])
        b_obj.add_review(review_obj)
        
    return b_obj
    # in the loop, add each individual parsed parsed business to our data container
    
    # return the container with all of the parsed results

In [8]:
class Business:
    def __init__(self, bus_id, name, price, rating, review_count):
        self.bus_id = bus_id
        self.name = name
        self.price = price
        self.rating = rating
        self.review_count = review_count
        self.reviews = []
        
    def add_review(self, review):
        self.reviews.append(review)
    def print_business(self):
        print(f"Id: {self.bus_id} name: {self.name} price: {self.price} rating: {self.rating} review count: {self.review_count}")
        for review in self.reviews:
            review.print_review()
    def test(self):
        print("In test")
    def insert_business_into_db(self, cnx, cursor):
        business_tuple = ()
        insert_sql = """INSERT INTO yelp_data.business 
        (business_id, business_name, business_rating, price, no_of_reviews) 
        VALUES (%s, %s, %s, %s, %s)"""
        business_tuple = (self.bus_id, self.name, self.rating, self.price, self.review_count)
        cursor.execute(insert_sql, business_tuple)
        cnx.commit()
        self.insert_reviews_into_db(cnx, cursor)      
    def insert_reviews_into_db(self, cnx, cursor):
        for review in self.reviews:
            review_tuple = ()
            insert_sql = """INSERT INTO yelp_data.reviews
            (review_id, rating, text, time_created, business_id)
            VALUES (%s, %s, %s, %s, %s)"""
            review_tuple = (review.id, review.rating, 
                            review.text, review.time_created, self.bus_id)

            cursor.execute(insert_sql, review_tuple)
            print(insert_sql, review_tuple)
            cnx.commit()


In [9]:


            
class Review:
    def __init__(self, id, text, rating, time_created):
        self.id = id
        self.text = text
        self.rating = rating
        self.time_created = time_created
    def print_review(self):
        print(f"Id: {self.id} review: {self.text} rating: {self.rating} timestamp: {self.time_created}")
        
        

        

In [22]:


def parse_business_and_reviews(offset):
    b_objs = parse_business(offset)
    for b_obj in b_objs:
        parse_review(b_obj)
        b_obj.print_business()
    return b_objs


In [25]:
# create a variable  to keep track of which result you are in. 
import time
def parse_pagination():
    num = data['total']
    pagesize = 20
    cur = 0
    b_objs = []
    val = input(f"Number of businesses is {num}, proceed [y|n]?")
    if val != 'y':
        return
    
    #set up a while loop to go through and grab the result 
    while cur < num and cur < 1000:
        #make your API call with the new offset number
        b_objs_page = parse_business_and_reviews(cur)
        b_objs.extend(b_objs_page)
        time.sleep(1)
        #increment the counter by 50 to move on to the next results
        cur += pagesize
        print(len(b_objs))
        
    for b_obj in b_objs:
        b_obj.insert_business_into_db(cnx, cursor)

In [1]:
#parse_pagination()

In [44]:
data = response.json()
print(data['total'])

129


In [15]:
business_list[0].print_business()

Id: c8FID4VcTtDDONokhACvfQ name: O'Lunney's price: $$ rating: 4.0 review count: 382
Id: dURIrDjOhV51M6aeWdw4cg review: We got the crab cake app the chicken pot pie and beef stew
We were told all is made in house
The stew was overflowing with goodies not a think soup like... rating: 4 timestamp: 2019-09-08 09:40:12
Id: ZBs4ZFSD8y37v_zQ28gXLA review: 2.5 Stars but I'll round up to 3.

We were looking for a quick meal before a show and decided to give O'Lunneys a try.  Their menu is kind of simple: some... rating: 3 timestamp: 2019-08-10 21:13:36
Id: AZhMNvOuHD-daAu86zSA1g review: PROS: This place was very nice! I walked in and it had a fancy atmosphere, attentive staff, and nice music playing in the back. This place has 3 sections: a... rating: 4 timestamp: 2019-07-13 17:54:35


In [14]:
business_list = parse_business(0)
parse_review(business_list[0])
business_list[0].print_business()


129
Id: c8FID4VcTtDDONokhACvfQ name: O'Lunney's price: $$ rating: 4.0 review count: 382
Id: dURIrDjOhV51M6aeWdw4cg review: We got the crab cake app the chicken pot pie and beef stew
We were told all is made in house
The stew was overflowing with goodies not a think soup like... rating: 4 timestamp: 2019-09-08 09:40:12
Id: ZBs4ZFSD8y37v_zQ28gXLA review: 2.5 Stars but I'll round up to 3.

We were looking for a quick meal before a show and decided to give O'Lunneys a try.  Their menu is kind of simple: some... rating: 3 timestamp: 2019-08-10 21:13:36
Id: AZhMNvOuHD-daAu86zSA1g review: PROS: This place was very nice! I walked in and it had a fancy atmosphere, attentive staff, and nice music playing in the back. This place has 3 sections: a... rating: 4 timestamp: 2019-07-13 17:54:35


In [16]:
business_list[0].insert_business_into_db(cnx, cursor)
#business_list[0].insert_reviews_into_db(cnx, cursor)
#dir(business_list[0])
#business_list[0].print_business()

INSERT INTO yelp_data.reviews
            (review_id, rating, text, time_created, business_id)
            VALUES (%s, %s, %s, %s, %s) ('dURIrDjOhV51M6aeWdw4cg', 4, 'We got the crab cake app the chicken pot pie and beef stew\nWe were told all is made in house\nThe stew was overflowing with goodies not a think soup like...', '2019-09-08 09:40:12', 'c8FID4VcTtDDONokhACvfQ')
INSERT INTO yelp_data.reviews
            (review_id, rating, text, time_created, business_id)
            VALUES (%s, %s, %s, %s, %s) ('ZBs4ZFSD8y37v_zQ28gXLA', 3, "2.5 Stars but I'll round up to 3.\n\nWe were looking for a quick meal before a show and decided to give O'Lunneys a try.  Their menu is kind of simple: some...", '2019-08-10 21:13:36', 'c8FID4VcTtDDONokhACvfQ')
INSERT INTO yelp_data.reviews
            (review_id, rating, text, time_created, business_id)
            VALUES (%s, %s, %s, %s, %s) ('AZhMNvOuHD-daAu86zSA1g', 4, 'PROS: This place was very nice! I walked in and it had a fancy atmosphere, attenti

In [9]:
def import_data():
    url = 'https://api.yelp.com/v3/businesses/search'
    client_id = 'V7jC9R0wg8iyi0uprqrIDQ'
    api_key = '__RrqPgntWvjGhyjuZrEW4sn6o3g-zID39CzFyAO8qkTIUgxGBvDdeB3_v3_qrBeCZ4QyimcSchxfoaQxx9LO-Kff6XNsoXvlFAXZMuOu66CyUS__M7xA4oZsSjpXXYx'

    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }

    term = 'Irish'
    location = 'Manhattan'
    categories = 'bars'
    status = True


    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'categories' : categories,
                    'open now' : status
                }

In [ ]:
# LIST OF THINGS WE NEED FOR TABLES
# business name, rating, price, review_text, review_date, number_of_reviews
# BUSINESS = business name, business_rating, price, number_of_reviews
# REVIEWS = review_text, review_rating, review_date

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [1]:
# Write a function to take your parsed data and insert it into the DB

In [ ]:
# Write a script that combines the three functions above into a single process.

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50